In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
import sys 

In [2]:
current_path = os.getcwd()
parent_path = os.path.abspath(os.path.dirname(os.getcwd()))

In [3]:
def get_custom_all(parent_path):

    year,imoex,value=[],[],[]

    for y in [2019,2020,2021,2022,2023,2024]:
        data=pd.read_excel(f'{parent_path}\\data\\全国海关数据\\{y}.xls')
        ex=data.loc[2,'Unnamed: 2']
        im=data.loc[2,'Unnamed: 4']
        exvalue=round(data.loc[4,'Unnamed: 3']/10000,2)
        imvalue=round(data.loc[4,'Unnamed: 5']/10000,2)
        
        year.append(y)
        year.append(y)
        imoex.append(im)
        imoex.append(ex)
        value.append(imvalue)
        value.append(exvalue)
        
    df_all=pd.DataFrame({'年份':year,'出口or进口':imoex,'全国总额':value})
    df_all_pt=pd.pivot_table(df_all,index=['年份'],values='全国总额',aggfunc=np.sum)
    df_all_pt['出口or进口']='进出口'
    df_all_pt=df_all_pt.reset_index()
    df_all=pd.concat([df_all,df_all_pt],axis=0,ignore_index=True)
    return df_all

In [4]:
def get_custom_sh(parent_path,df_all):

    year,imex,value,percent=[],[],[],[]

    dir_list = os.listdir(parent_path+'\\data\\上海海关分贸易方式')
    for dirname in dir_list:
        data=pd.read_excel(parent_path+'\\data\\上海海关分贸易方式\\'+dirname)
        y=dirname[0:4]
        year += [y]*3
        imex = imex + ['进出口','出口','进口']
        
        if y in ['2021','2022','2023','2024']:
            percent.append(round(data.loc[3,'Unnamed: 4'],2))
            percent.append(round(data.loc[3,'Unnamed: 8'],2))
            percent.append(round(data.loc[3,'Unnamed: 12'],2))
            
            value.append(round(data.loc[3,'Unnamed: 3'],2))
            value.append(round(data.loc[3,'Unnamed: 7'],2))
            value.append(round(data.loc[3,'Unnamed: 11'],2))
            
        else:
            percent.append(round(data.loc[3,'Unnamed: 2'],2))
            percent.append(round(data.loc[3,'Unnamed: 6'],2))
            percent.append(round(data.loc[3,'Unnamed: 10'],2))
            
            value.append(round(data.loc[3,'Unnamed: 1'],2))
            value.append(round(data.loc[3,'Unnamed: 5'],2))
            value.append(round(data.loc[3,'Unnamed: 9'],2))

    df_trade=pd.DataFrame({"年份":year,'进口or出口':imex,'金额':value,'同比':percent})
    df_all['年份']=df_all['年份'].astype('str')
    res=df_all.merge(df_trade,left_on=['出口or进口','年份'],right_on=['进口or出口','年份'],how='left')
    res['金额']=round(res['金额'],2)
    res.loc[res['金额']>10000000,'金额'] = res.loc[res['金额']>10000000,'金额']/10000
    res['上海占比全国']=res['金额']/res.全国总额*100
    res.sort_values(by=['出口or进口'],ascending=[True],inplace=True)
    res.drop('进口or出口',axis=1,inplace=True)
    return res 


In [5]:
df_all = get_custom_all(parent_path)
res = get_custom_sh(parent_path,df_all)

In [6]:
res

,年份,出口or进口,全国总额,金额,同比,上海占比全国
1,2019,出口,172342.34,13720.910000,0.41,7.961427
3,2020,出口,179326.36,13725.360000,0.00,7.653844
5,2021,出口,217347.60,15718.670000,14.56,7.232042
7,2022,出口,239654.00,17134.210000,9.04,7.149561
9,2023,出口,237725.88,17377.940609,1.62,7.310075
11,2024,出口,99501.59,7062.140282,1.04,7.097515
17,2024,进出口,175042.46,17491.862702,0.79,9.992926
15,2022,进出口,420678.16,41902.750000,3.20,9.960762
14,2021,进出口,391008.54,40610.350000,16.45,10.386052
13,2020,进出口,321556.93,34828.470000,2.27,10.831199


In [7]:
res.to_excel(r'../output/2.1结果.xlsx')

In [8]:
res.to_excel('上海结果.xlsx',index=False)